In [ ]:
# Run this cell
import pandas as pd

In [ ]:
# Run this cell
df = pd.read_csv("../datasets/people/people.csv", sep="\t")
df["oppstartsdato"] = pd.to_datetime(df["oppstartsdato"])
df.index.name = "ansattnr"
df.head()

In [ ]:
# Run this cell
# This will give our employees a salary
import numpy as np
dfsort = df.sort_values(["senioritet", "oppstartsdato", "avdeling"], ascending=[True, False, False])
payroll = pd.Series([400000+2000*i for i in range(100)], index=dfsort.index)
payroll += np.random.randint(
    0, 10000, size=len(payroll))
payroll.name = "lønn"
df = df.join(payroll)

## Selection

In [ ]:
# Return the epost of employees with ansattnr 45-48

# Your code goes here!
df.loc[45:48, "epost"]

In [ ]:
# Find males employes have ansvarsdag on Tuesdays

df.loc[(df.kjønn == "han") & (df.ansvarsdag == "tirsdag")]

In [ ]:
# Find the employes who started between 2018.05.05-2019.04.13

# hint: It's easier if we set the index first. Just remember not to change df itself
df.set_index("oppstartsdato").loc["2018.05.05":"2019.04.13"]

In [ ]:
# Hi!
# I am your boss here at Company Ltd, Torgrim Ruud.
# You can call me Mr. Ruud, or you can be fired.

# People here at the office don't appreciate that I give my family preferential treatment.
# But what is so wrong with nepotism? The apple rarely falls far from the tree, am I right?

# Task:
# Provide me a list of emails (and names) that share my last name, so that I can send them some shares.

# hint: https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing

# Your code goes here!
df.loc[df.etternavn == "Ruud", ["fornavn", "etternavn", "epost"]]

In [ ]:
# Complaining is never good! I have sent out a varsel to everyone who answered negatively on our HR survey.
# Staff who has received a varsel should be taught a message. Give them a 20% pay cut!

# You can use this graph to check if your salary slash was successful.

# Run this cell, then write your solution (salary slash code) in the next cell

df.groupby("varslet").lønn.mean().plot(kind="bar", legend=True, title="Before punishment");

In [ ]:
# Salary slash code code goes here!
df.loc[df.varslet == True, "lønn"] *= 0.9

# Plotting to check if it worked
df.groupby("varslet").lønn.mean().plot(kind="bar", legend=True, title="After punishment");

In [ ]:
# My birthday is on November 19th and I want to keep the attention on me. I want you to find those selfish people 
# who decided to have their work anniversary on my special day.. And get rid of them! 

# (He first wants you to find people with oppstartsdato the same date as him)

selfish = df.loc[(df.oppstartsdato.dt.month == 11) & (df.oppstartsdato.dt.day == 19)]
selfish

In [ ]:
# (Then he wants you to lay them off, i.e. drop them from our dataframe)

# hint: DataFrame.drop just needs to know which index labels to drop

df.drop(selfish.index, inplace=True)

In [ ]:
# I don't want my underlings snooping into my personal business at home. I have plenty of things to hide!
# I live at "3769 Kristinhamn" and if any staff lives there, give them the boot!

# hint: .str accessor and DataFrame.drop

snoopers = df[df.addresse.str.contains("3769 Kristinhamn")]
print(snoopers[["fornavn", "etternavn", "addresse"]])
df.drop(snoopers.index, inplace=True)

In [ ]:
# Give me a list of all people who share last name with someone else. I don't like families working together.

# hint: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

df[df.etternavn.duplicated(keep=False)].sort_values("etternavn")

In [ ]:
# Plot lønn (y) as a function of oppstartsdato (x)
df.plot(y="lønn", x="oppstartsdato");

In [ ]:
# Why does this not work the same as the cell above?
# Here we try to sort the values first, then plot without using y="oppstartsdato"
df_sorted = df.sort_values("oppstartsdato")

# When we print the dataframe, it sure looks like the rows are sorted by oppstartsdato!
print(df_sorted[["oppstartsdato", "lønn"]].head())

# Then why does the plot look like a Picasso painting?
df_sorted.plot(y="lønn");

In [ ]:
# Try to the lønn again (against oppstartsdato), but use a scatter plot this time
df.plot(kind="scatter", y="lønn", x="oppstartsdato");

In [ ]:
# This cell showcases two things:
# 1. that you can make a regression curve using numpy
# 2. that you can have multiple plots in a single "box"

# Don't feel the need to understand this code by the way.
# For now you can just run it and have a look at the plots.
# What does the plots tell us about how salary is distributed?

import numpy as np

def regr(df, x_axis, title=""):
    """
    Plots a linear regression line of "lønn" column based on x_axis
    """
    df = df.copy()
    df["x_axis"] = x_axis
    
    # Finding the linear regression line (ax + b)
    d = np.polyfit(df["x_axis"], df.lønn, 1)
    f = np.poly1d(d)

    print(f"Regression curve {title}: {f}")
    
    regression = f(df["x_axis"])

    df = df.assign(regression=regression)
    
    # Two plots in one box
    ax = df.plot(kind="scatter", y='lønn', x="x_axis", title=title)
    df.plot(y='regression', x="x_axis", color='Red', ax=ax);
    

regr(df, df.oppstartsdato.dt.year, "Starting year")
regr(df, df.senioritet, "Seniority")

# Groupby, plotting

In [ ]:
# Using groupby, plot the average salary (lønn) as a bar chart for each department (avdeling)

df.groupby("avdeling").lønn.mean().plot(kind="bar");

In [ ]:
# Are you able to plot it again, but with the bars sorted by height?

df.groupby("avdeling").lønn.mean().sort_values().plot(kind="bar");

In [ ]:
# Plot lønn by senioritet (one line for each senioritet)
# Use df.groupby()
# Hint: when plotting, use legend=True
df.groupby("senioritet").lønn.plot(legend=True);

In [ ]:
# Using groupby, plot the median salary for each oppstartdato year

# hint: you can group by df.oppstartsdato.dt.year
df.groupby(df.oppstartsdato.dt.year).lønn.median().plot();

In [ ]:
# grouping by year started, find the last person to start for each year
latest = df.groupby(df.oppstartsdato.dt.year).oppstartsdato.idxmin()
df.loc[latest, ["fornavn", "etternavn", "oppstartsdato"]]

In [ ]:
# Torgrim Ruud has a list of company policies that he demands you check, one by one.
# Here follows:

In [ ]:
# Policy compliance check:
# Average seniority for every department must be over 3.0
df.groupby("avdeling").senioritet.mean()

In [ ]:
# Policy compliance check:
# There must be at least 1 sertified person for each ansvarsdag
df.groupby("avdeling").sertifisert.sum()

In [ ]:
# Policy compliance check:
# every avdeling must have at least one person with max seniority (6).
df.groupby(df.avdeling).senioritet.max() >= 6

In [ ]:
# Policy compliance check:
# every seniority in a given avdeling must have at least one person who is certified.
df.groupby([df.avdeling, df.senioritet]).sertifisert.sum() >= 1

In [ ]:
# Policy compliance check:
# On ansvarsdag: Saturday, all attendes must be without a varsel
# hint: use Grouper.get_group(). This one is a two-liner.
gr = df.groupby("ansvarsdag")
gr.get_group("lørdag").varslet.any()

In [ ]:
# Within each department, the man and woman who has stayed the longest with the Company shall receive an annual price.
# make a list of people to celebrate for this years party.
longest = df.groupby(["avdeling", "kjønn"]).oppstartsdato.idxmin()
df.loc[longest.values, ["fornavn", "etternavn", "oppstartsdato"]]

In [ ]:
# Using Grouper.apply(), find the average senioritet for each avdeling.
df.groupby("avdeling").apply(lambda group: group.senioritet.mean())

In [ ]:
# For every seniority level at least one person must have the number "1" in their brukernavn
# Use Grouper.apply()
df.groupby("senioritet").apply(lambda group: group.brukernavn.str.contains("1").any())

In [ ]:
# Challenging
# Policy compliance check:
# No avdeling is allowed to employ people who share last names (because it causes our computer system to crash!)
# hint: use .groupby() and .apply()
df.groupby("avdeling").apply(lambda group: group[group.etternavn.duplicated(keep=False)])

In [ ]:
# Challenging
# Policy compliance check:
# every ansvarsdag must have at least one person who has stayed in the company for 3 years or longer.
# hint: here we need to do with timeseries subtraction
(pd.Timestamp.now() - df.groupby(df.ansvarsdag).oppstartsdato.max()).dt.days / 365